In [26]:
#爬布雷克網站用的
# {
#     'Page-Title' : {
#         'Title' : 'Page-Title',
#         'Type' : '食|衣|住|行|生活|育樂|醫療|養生',
#         'Author' : '網誌作者，如果有的話，沒有的話存成 無 ',
#         'Date' : '文章發布日期',
#         'Body' : '要用的文章，後續斷句使用',
#         'Url' : '文章的連結'
#     },
# }
import requests
import json
from bs4 import BeautifulSoup
tag_list=[]
Page_Title={}
# Page_data={'Title':'','Type':'','Author':'無','Date' : '文章發布日期','Body' : '要用的文章，後續斷句使用','Url' : '文章的連結'}

def Set_Data(url,Type,tag_list):
    href=[]
    Title=''
    Author=''
    Date=''
    text=''
    Number=1
    count=0
    while (True):
        r=requests.get(url+str(Number))
        Number+=1
        soup=BeautifulSoup(r.text,"html.parser")
        sel = soup.select("div.container div#content h2 a") #各文章的標題連結html
        if (not sel): break
        for i in sel:
            text='' #初始化合併過的內文
            page=i.get('href')
            page_r=requests.get(page) #開啟前面select/get到的網址
            page_soup=BeautifulSoup(page_r.text,"html.parser")
            
            tag_check=False
            for a in page_soup.select("div.post-header span.cat a"): #若標籤存在於排除列表，則忽略此篇文章
                if (a.text in tag_list):
                    tag_check=True
                    break
            if (tag_check):continue
                
            Title=page_soup.select("div.post-header h1")[0].text
            Author=page_soup.select("div.author-content a")[0].text
            Date=page_soup.select("div.post-header span.post-date")[0].text
            Body_P=page_soup.select("div.post-entry p")
            for p in Body_P: #用以合併所有文本，排除圖片
                if not p.find('img'):
                    text+=p.text
            Page_data={'Title':Title,'Type':Type,'Author':Author,'Date' : Date[1:],'Body' : text}
            Page_Title[Title]=Page_data
            
            count+=1
            print("NO."+str(count)+" "+Title)
            
while(1 ):
    print("---貼上要爬蟲的頁面的第一頁，注意網址要顯示格式為....../page/，輸入為空則結束\n---",end='')
    url=input()
    if (not url): break
    print("---輸入爬取的網頁資料類型，食|衣|住|行|生活|育樂|醫療|養生\n---",end='')
    Type=input()
    print("---輸入要忽略的文章tag，輸入一個後enter輸入下個，輸入為空則輸入完畢\n---",end='')
    while (True):
        tag_add=input()
        if (tag_add):
            tag_list.append(tag_add)
            print("---",end='')
        else:
            break
    print("---開始爬取---")
    Set_Data(url,Type,tag_list)

---貼上要爬蟲的頁面的第一頁，注意網址要顯示格式為....../page/，輸入為空則結束
---

KeyboardInterrupt: Interrupted by user

In [27]:
print("輸入創建的檔名，預設為'Crawl_from_Blake'\n",end='')
name=input()
if (not name):
    name="Crawl_from_Blake"
    print(name)
with open(name+".json","w",encoding='utf-8')as f:
    json.dump(Page_Title,f,indent=4,ensure_ascii=False) #存檔
print("---結束---")

輸入創建的檔名，預設為'Crawl_from_Blake'
test
---結束---
